In [1]:
import regex as re
import gc
import time
import numpy as np
import pandas as pd
import pickle
import seaborn as sns
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer, KNNImputer
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

2023-04-02 13:09:13.786303: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Ordinal Encoding

In [2]:
def apply_ordinal_encoding(train_data, test_data, categorical_features):
    '''
    Apply ordinal encoding to categorical features in train and test data
    '''
    # Initialize the encoder
    encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
    
    # Fit the encoder on the train data
    encoder.fit(train_data[categorical_features])
    
    # Transform the train and test data using the encoder
    train_data_encoded = train_data.copy()
    train_data_encoded[categorical_features] = encoder.transform(train_data[categorical_features])
    
    test_data_encoded = test_data.copy()
    test_data_encoded[categorical_features] = encoder.transform(test_data[categorical_features])
    
    return train_data_encoded, test_data_encoded

In [8]:
def cat_num_features(df):
    
    '''
        Utility Function to get the names of Categorical Features and 
        Numerical Features of the given Dataset.
    '''
    
    catf = []
    numf = []
    
    catf = [
            'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 
            'card6', 'addr1', 'addr2', 'P_emaildomain', 'R_emaildomain', 'M1', 'M2', 
            'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'DeviceType', 'DeviceInfo',
            'TransactionDW', 'card1_div_1000', 'card2_div_10', 'P_parent_domain', 
            'P_domain_name', 'P_top_level_domain', 'R_parent_domain', 'R_domain_name', 
            'R_top_level_domain', 'device_name', 'device_version', 'os_name', 'os_version', 
            # 'screen_width', 'screen_height', 'card_intr1', 'card_intr2', 'card1_addr1', 
            # 'card1_addr2', 'card2_addr1', 'card2_addr2', 'card3_addr1', 'card3_addr2', 
            # 'card5_addr1', 'card5_addr2', 'card6_addr1', 'card6_addr2', 'ProductCD_addr1', 
            # 'ProductCD_addr2', 'card1_ProductCD', 'card2_ProductCD', 'card5_ProductCD', 
            # 'card6_ProductCD', 'addr1_P_emaildomain', 'card1_P_emaildoman', 'card1_addr1_P_emaildomain',
            # 'uid1', 'uid2'
            ]

    catf+=['id_'+str(i) for i in range(12,39)]


    # Updating the Categorical Feature Names List based on the columns present in the dataframe
    catf = [feature for feature in catf if feature in df.columns]
    numf = [feature for feature in df.columns if feature not in catf and not feature == 'isFraud']
    
    return (catf, numf)  

In [9]:
def label_encode(X_train, X_test, catf):
  
  '''
    Utility Function to Encode Categorical Features.
  '''

  for f in catf:
    
    X_train[f] = X_train[f].astype(str)
    X_test[f] = X_test[f].astype(str)
    
    le = LabelEncoder()
    le.fit(X_train[f])
    mapping = dict(zip(le.classes_, le.transform(le.classes_)))
    X_train[f] = le.transform(X_train[f])
    
    # Manually Encoding the test and Test Dataset so as to avoid error for any category which is not present in train set
    
    # All the categories which are not present in train datset are encoded as -1    
    X_test[f] = [-1 if mapping.get(v, -1)==-1 else mapping[v] for v in X_test[f].values ]

  return (X_train, X_test)

In [4]:
%%time
train_data = pd.read_pickle('basic_fe_train.pkl')
test_data = pd.read_pickle('basic_fe_test.pkl')

CPU times: user 1.25 s, sys: 1.1 s, total: 2.35 s
Wall time: 2.59 s


In [69]:
test_data.head()

,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,P_emaildomain,R_emaildomain,C1,C2,C4,C5,C6,C7,C9,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V3,V4,V6,V8,V11,V13,V18,V20,V30,V36,V37,V40,V44,V47,V48,V52,V54,V60,V70,V76,V78,V80,V81,V82,V87,V91,V99,V127,V130,V131,V139,V147,V156,V165,V166,V169,V171,V173,V174,V175,V176,V180,V183,V185,V195,V198,V203,V205,V207,V209,V210,V215,V216,V220,V221,V223,V230,V234,V235,V238,V240,V241,V250,V252,V257,V258,V260,V262,V263,V264,V265,V268,V271,V274,V277,V282,V283,V289,V291,V303,V307,V310,V312,V315,id_01,id_02,id_03,id_04,id_05,id_06,id_09,id_10,id_11,id_12,id_13,id_15,id_16,id_17,id_19,id_20,id_28,id_29,id_30,id_33,id_35,id_36,id_37,DeviceType,DeviceInfo,TransactionHR,TransactionMT,TransactionDW,dollars,cents,P_parent_domain,P_domain_name,P_top_level_domain,card1_div_1000,card2_div_10,R_parent_domain,R_domain_name,R_top_level_domain,device_name,device_version,os_name,os_version
TransactionID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3663549,18403224,31.95,-1,458,-1,-1,-1,-1,-1,-1,-1,1.0,-1,-1,6.0,6.0,0.0,3.0,4.0,0.0,6.0,5.0,1.0,115.0,6.0,419.0,419.0,27.0,398.0,27.0,NaN,NaN,NaN,418.0,203.0,NaN,NaN,NaN,409.0,-1,-1,-1,-1,-1,-1,-1,-1,-1,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,47.950001,47.950001,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,0.0,47.950001,47.950001,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,0,31,95,-1,-1,-1,2,-1,-1,-1,-1,-1,-1,-1,-1
3663550,18403263,49.00,-1,7191,-1,-1,-1,-1,-1,-1,-1,4.0,-1,-1,3.0,2.0,0.0,0.0,1.0,0.0,2.0,1.0,1.0,12.0,2.0,149.0,149.0,7.0,634.0,7.0,NaN,NaN,NaN,231.0,634.0,NaN,NaN,NaN,634.0,-1,-1,-1,-1,-1,-1,-1,-1,-1,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,4.0,280.000000,280.000000,77.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,0.0,280.000000,280.000000,77.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,1,0,49,0,-1,-1,-1,12,-1,-1,-1,-1,-1,-1,-1,-1
3663551,18403310,171.00,-1,7417,-1,-1,-1,-1,-1,-1,-1,2635.0,-1,-1,2.0,2.0,0.0,0.0,5.0,0.0,4.0,2.0,0.0,22.0,2.0,137.0,137.0,10.0,97.0,10.0,NaN,NaN,NaN,136.0,136.0,NaN,NaN,NaN,97.0,-1,-1,-1,-1,-1,-1,-1,-1,-1,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,1.0,1.0,0.0,3.0,968.000000,705.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,2.0,0.0,1321.000000,1058.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,1,0,171,0,-1,-1,-1,12,-1,-1,-1,-1,-1,-1,-1,-1
3663552,18403310,284.95,-1,1101,-1,-1,-1,-1,-1,-1,-1,17.0,-1,-1,5.0,2.0,0.0,1.0,1.0,0.0,2.0,2.0,0.0,7.0,4.0,42.0,42.0,41.0,242.0,41.0,NaN,NaN,NaN,242.0,242.0,NaN,NaN,NaN,242.0,-1,-1,-1,-1,-1,-1,-1,-1,-1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,2.0,2.0,0.0,2.0,2.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,0.0,0.000000,0.000000,0.000000,282.540009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1,-1,-1,-1,-1,-1,-1,-1

In [5]:
pd.__version__ 

'1.5.2'

In [6]:
train_data.head()

,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,P_emaildomain,R_emaildomain,C1,C2,C4,C5,C6,C7,C9,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V3,V4,V6,V8,V11,V13,V18,V20,V30,V36,V37,V40,V44,V47,V48,V52,V54,V60,V70,V76,V78,V80,V81,V82,V87,V91,V99,V127,V130,V131,V139,V147,V156,V165,V166,V169,V171,V173,V174,V175,V176,V180,V183,V185,V195,V198,V203,V205,V207,V209,V210,V215,V216,V220,V221,V223,V230,V234,V235,V238,V240,V241,V250,V252,V257,V258,V260,V262,V263,V264,V265,V268,V271,V274,V277,V282,V283,V289,V291,V303,V307,V310,V312,V315,id_01,id_02,id_03,id_04,id_05,id_06,id_09,id_10,id_11,id_12,id_13,id_15,id_16,id_17,id_19,id_20,id_28,id_29,id_30,id_33,id_35,id_36,id_37,DeviceType,DeviceInfo,TransactionHR,TransactionMT,TransactionDW,dollars,cents,P_parent_domain,P_domain_name,P_top_level_domain,card1_div_1000,card2_div_10,R_parent_domain,R_domain_name,R_top_level_domain,device_name,device_version,os_name,os_version
TransactionID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,credit,315.0,87.0,19.0,NaN,NaN,1.0,1.0,0.0,0.0,1.0,0.0,1.0,2.0,0.0,1.0,1.0,14.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,13.0,13.0,NaN,NaN,NaN,0.0,T,T,T,M2,F,T,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,117.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,1.0,0.0,117.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,5,68,5,NaN,NaN,NaN,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,325.0,87.0,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,M0,T,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,5,29,0,google,gmail,com,2,40.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,debit,330.0,87.0,287.0,outlook.com,NaN,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,315.0,NaN,NaN,NaN,315.0,T,T,T,M0,F,F,F,F,F,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,5,59,0,microsoft,outlook,com,4,49.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,debit,476.0,87.0,NaN,yahoo.com,NaN,2.0,5.0,0.0,0.0,4.0,0.0,1.0,1.0,0.0,25.0,1.0,112.0,112.0,0.0,94.0,0.0,NaN,NaN,NaN,84.0,NaN,NaN,NaN,NaN,111.0,NaN,NaN,NaN,M0,T,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,10.0,1758.0,354.0,135.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,0.0,1758.0,354.0,135.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [10]:
catf, numf = cat_num_features(train_data)

# Data Prep

In [11]:
X_train = train_data.drop(['isFraud'], axis=1)
y_train = train_data['isFraud']

X_test = test_data

In [12]:
del train_data, test_data

In [13]:
# Storing Categorical and Numerical Feature Names 

catf, numf = cat_num_features(X_train)

In [14]:
# Encoding the Categorical Features

X_train[catf] = X_train[catf].fillna('missing')
X_test[catf] = X_test[catf].fillna('missing')

X_train, X_test = label_encode(X_train, X_test, catf)

In [15]:
X_train.head()

,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,P_emaildomain,R_emaildomain,C1,C2,C4,C5,C6,C7,C9,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V3,V4,V6,V8,V11,V13,V18,V20,V30,V36,V37,V40,V44,V47,V48,V52,V54,V60,V70,V76,V78,V80,V81,V82,V87,V91,V99,V127,V130,V131,V139,V147,V156,V165,V166,V169,V171,V173,V174,V175,V176,V180,V183,V185,V195,V198,V203,V205,V207,V209,V210,V215,V216,V220,V221,V223,V230,V234,V235,V238,V240,V241,V250,V252,V257,V258,V260,V262,V263,V264,V265,V268,V271,V274,V277,V282,V283,V289,V291,V303,V307,V310,V312,V315,id_01,id_02,id_03,id_04,id_05,id_06,id_09,id_10,id_11,id_12,id_13,id_15,id_16,id_17,id_19,id_20,id_28,id_29,id_30,id_33,id_35,id_36,id_37,DeviceType,DeviceInfo,TransactionHR,TransactionMT,TransactionDW,dollars,cents,P_parent_domain,P_domain_name,P_top_level_domain,card1_div_1000,card2_div_10,R_parent_domain,R_domain_name,R_top_level_domain,device_name,device_version,os_name,os_version
TransactionID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2987000,86400,68.5,4,3417,500,42,1,38,1,166,65,19.0,31,31,1.0,1.0,0.0,0.0,1.0,0.0,1.0,2.0,0.0,1.0,1.0,14.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,13.0,13.0,NaN,NaN,NaN,0.0,1,1,1,2,0,1,2,2,2,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,117.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,1.0,0.0,117.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,54,3,2,104,522,394,2,2,74,260,2,2,2,1,1735,0,0,5,68,5,16,24,7,4,51,16,24,8,13,293,7,68
2987001,86401,29.0,4,7922,303,42,2,2,1,173,65,NaN,16,31,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,2,2,2,0,1,1,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,54,3,2,104,522,394,2,2,74,260,2,2,2,1,1735,0,0,5,29,0,12,15,2,10,30,16,24,8,13,293,7,68
2987002,86469,59.0,4,9383,389,42,4,58,2,178,65,287.0,36,31,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,315.0,NaN,NaN,NaN,315.0,1,1,1,0,0,0,0,0,0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,54,3,2,104,522,394,2,2,74,260,2,2,2,1,1735,0,1,5,59,0,15,28,2,12,39,16,24,8,13,293,7,68
2987003,86499,50.0,4,6991,466,42,2,14,2,282,65,NaN,54,31,2.0,5.0,0.0,0.0,4.0,0.0,1.0,1.0,0.0,25.0,1.0,112.0,112.0,0.0,94.0,0.0,NaN,NaN,NaN,84.0,NaN,NaN,NaN,NaN,111.0,2,2,2,0,1,0,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,10.0,1758.0,354.0,135.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,0.0,1758.0,354.0,135.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,54,3,2,104,522,394,2,2,74,260,2,2,2,1,1735,0,1,5,50,0,30,43,2,9,46,16,24,8,13,293,7,68
2987004,86506,50.0,1,9262,413,42,2,2,1,241,65,NaN,16,31,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2,2,3,2,2,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN

## Filling NaN with -1s

In [16]:
X_train.fillna(-1, inplace=True)

In [17]:
X_test.fillna(-1, inplace=True)

In [18]:
# Save data for future use
X_train.to_pickle('train_clean_neg1.pkl')
X_test.to_pickle('test_clean_neg1.pkl')

## Imputing using KNN

In [72]:
%%time
train_data = pd.read_pickle('basic_fe_train.pkl')
test_data = pd.read_pickle('basic_fe_test.pkl')

CPU times: user 2.04 s, sys: 1.65 s, total: 3.69 s
Wall time: 4.43 s


In [73]:
X_train = train_data.drop(['isFraud'], axis=1)
y_train = train_data['isFraud']

X_test = test_data

In [74]:
# Encoding the Categorical Features

X_train[catf] = X_train[catf].fillna('missing')
X_test[catf] = X_test[catf].fillna('missing')

X_train, X_test = label_encode(X_train, X_test, catf)

In [75]:
catf, numf = cat_num_features(X_train)

In [76]:
train_sample_10k = X_train.iloc[:10000]
test_sample_10k = X_test.iloc[:10000]

In [77]:
train_sample_10k.head()

,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,P_emaildomain,R_emaildomain,C1,C2,C4,C5,C6,C7,C9,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V3,V4,V6,V8,V11,V13,V18,V20,V30,V36,V37,V40,V44,V47,V48,V52,V54,V60,V70,V76,V78,V80,V81,V82,V87,V91,V99,V127,V130,V131,V139,V147,V156,V165,V166,V169,V171,V173,V174,V175,V176,V180,V183,V185,V195,V198,V203,V205,V207,V209,V210,V215,V216,V220,V221,V223,V230,V234,V235,V238,V240,V241,V250,V252,V257,V258,V260,V262,V263,V264,V265,V268,V271,V274,V277,V282,V283,V289,V291,V303,V307,V310,V312,V315,id_01,id_02,id_03,id_04,id_05,id_06,id_09,id_10,id_11,id_12,id_13,id_15,id_16,id_17,id_19,id_20,id_28,id_29,id_30,id_33,id_35,id_36,id_37,DeviceType,DeviceInfo,TransactionHR,TransactionMT,TransactionDW,dollars,cents,card1_div_1000,card2_div_10,P_parent_domain,P_domain_name,P_top_level_domain,R_parent_domain,R_domain_name,R_top_level_domain,device_name,device_version,os_name,os_version
TransactionID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2987000,86400,68.5,4,3417,500,42,1,38,1,166,65,19.0,31,31,1.0,1.0,0.0,0.0,1.0,0.0,1.0,2.0,0.0,1.0,1.0,14.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,13.0,13.0,NaN,NaN,NaN,0.0,1,1,1,2,0,1,2,2,2,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,117.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,1.0,0.0,117.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,54,3,2,104,522,394,2,2,74,260,2,2,2,1,1735,0,0,5,68,5,4,51,16,24,7,16,24,8,13,293,7,68
2987001,86401,29.0,4,7922,303,42,2,2,1,173,65,NaN,16,31,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,2,2,2,0,1,1,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,54,3,2,104,522,394,2,2,74,260,2,2,2,1,1735,0,0,5,29,0,10,30,12,15,2,16,24,8,13,293,7,68
2987002,86469,59.0,4,9383,389,42,4,58,2,178,65,287.0,36,31,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,315.0,NaN,NaN,NaN,315.0,1,1,1,0,0,0,0,0,0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,54,3,2,104,522,394,2,2,74,260,2,2,2,1,1735,0,1,5,59,0,12,39,15,28,2,16,24,8,13,293,7,68
2987003,86499,50.0,4,6991,466,42,2,14,2,282,65,NaN,54,31,2.0,5.0,0.0,0.0,4.0,0.0,1.0,1.0,0.0,25.0,1.0,112.0,112.0,0.0,94.0,0.0,NaN,NaN,NaN,84.0,NaN,NaN,NaN,NaN,111.0,2,2,2,0,1,0,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,10.0,1758.0,354.0,135.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,0.0,1758.0,354.0,135.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,54,3,2,104,522,394,2,2,74,260,2,2,2,1,1735,0,1,5,50,0,9,46,30,43,2,16,24,8,13,293,7,68
2987004,86506,50.0,1,9262,413,42,2,2,1,241,65,NaN,16,31,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2,2,3,2,2,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN

In [78]:
test_sample_10k.head()

,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,P_emaildomain,R_emaildomain,C1,C2,C4,C5,C6,C7,C9,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V3,V4,V6,V8,V11,V13,V18,V20,V30,V36,V37,V40,V44,V47,V48,V52,V54,V60,V70,V76,V78,V80,V81,V82,V87,V91,V99,V127,V130,V131,V139,V147,V156,V165,V166,V169,V171,V173,V174,V175,V176,V180,V183,V185,V195,V198,V203,V205,V207,V209,V210,V215,V216,V220,V221,V223,V230,V234,V235,V238,V240,V241,V250,V252,V257,V258,V260,V262,V263,V264,V265,V268,V271,V274,V277,V282,V283,V289,V291,V303,V307,V310,V312,V315,id_01,id_02,id_03,id_04,id_05,id_06,id_09,id_10,id_11,id_12,id_13,id_15,id_16,id_17,id_19,id_20,id_28,id_29,id_30,id_33,id_35,id_36,id_37,DeviceType,DeviceInfo,TransactionHR,TransactionMT,TransactionDW,dollars,cents,card1_div_1000,card2_div_10,P_parent_domain,P_domain_name,P_top_level_domain,R_parent_domain,R_domain_name,R_top_level_domain,device_name,device_version,os_name,os_version
TransactionID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3663549,18403224,31.95,4,365,10,42,4,108,2,52,65,1.0,16,31,6.0,6.0,0.0,3.0,4.0,0.0,6.0,5.0,1.0,115.0,6.0,419.0,419.0,27.0,398.0,27.0,NaN,NaN,NaN,418.0,203.0,NaN,NaN,NaN,409.0,1,1,0,3,2,0,1,1,1,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,47.950001,47.950001,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,0.0,47.950001,47.950001,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,54,3,2,104,522,394,2,2,74,260,2,2,2,1,1735,0,0,0,31,95,1,1,12,15,2,16,24,8,13,293,7,68
3663550,18403263,49.00,4,9099,10,42,4,108,2,151,65,4.0,2,31,3.0,2.0,0.0,0.0,1.0,0.0,2.0,1.0,1.0,12.0,2.0,149.0,149.0,7.0,634.0,7.0,NaN,NaN,NaN,231.0,634.0,NaN,NaN,NaN,634.0,1,0,0,0,2,0,2,2,2,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,4.0,280.000000,280.000000,77.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,0.0,280.000000,280.000000,77.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,54,3,2,104,522,394,2,2,74,260,2,2,2,1,1735,0,1,0,49,0,12,1,2,2,2,16,24,8,13,293,7,68
3663551,18403310,171.00,4,9245,473,42,4,108,2,280,65,2635.0,19,31,2.0,2.0,0.0,0.0,5.0,0.0,4.0,2.0,0.0,22.0,2.0,137.0,137.0,10.0,97.0,10.0,NaN,NaN,NaN,136.0,136.0,NaN,NaN,NaN,97.0,1,1,0,0,0,0,0,0,0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,1.0,1.0,0.0,3.0,968.000000,705.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,2.0,0.0,1321.000000,1058.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,54,3,2,104,522,394,2,2,74,260,2,2,2,1,1735,0,1,0,171,0,12,47,15,17,2,16,24,8,13,293,7,68
3663552,18403310,284.95,4,870,259,42,4,58,2,79,65,17.0,16,31,5.0,2.0,0.0,1.0,1.0,0.0,2.0,2.0,0.0,7.0,4.0,42.0,42.0,41.0,242.0,41.0,NaN,NaN,NaN,242.0,242.0,NaN,NaN,NaN,242.0,1,1,1,3,2,1,2,2,2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,2.0,2.0,0.0,2.0,2.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,0.0,0.000000,0.000000,0.000000,282.540009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,54,3,2,104,522,394,2,2,74,260,2,2,2,1,1735,0,1,0,284,95,1,26,12,15,2,16,24,8,13,293,7,

In [38]:
knn2 = KNNImputer(n_neighbors=2)

In [39]:
knn2.fit(train_sample_10k[numf])

KNNImputer(n_neighbors=2)

In [79]:
knn2_test = KNNImputer(n_neighbors=2)

In [80]:
knn2_test.fit(test_sample_10k[numf])

KNNImputer(n_neighbors=2)

In [40]:
train_sample_10k[numf] = knn2.transform(train_sample_10k[numf])

In [41]:
train_sample_10k[numf].head()

,TransactionDT,TransactionAmt,dist1,C1,C2,C4,C5,C6,C7,C9,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D8,D9,D10,D11,D12,D13,D14,D15,V1,V3,V4,V6,V8,V11,V13,V18,V20,V30,V36,V37,V40,V44,V47,V48,V52,V54,V60,V70,V76,V78,V80,V81,V82,V87,V91,V99,V127,V130,V131,V139,V147,V156,V165,V166,V169,V171,V173,V174,V175,V176,V180,V183,V185,V195,V198,V203,V205,V207,V209,V210,V215,V216,V220,V221,V223,V230,V234,V235,V238,V240,V241,V250,V252,V257,V258,V260,V262,V263,V264,V265,V268,V271,V274,V277,V282,V283,V289,V291,V303,V307,V310,V312,V315,id_01,id_02,id_03,id_04,id_05,id_06,id_09,id_10,id_11,TransactionHR,TransactionMT,dollars,cents
TransactionID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2987000,86400.0,68.5,19.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,2.0,0.0,1.0,1.0,14.0,59.0,13.0,0.0,10.5,0.0,23.5,0.0,13.0,13.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.5,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,5155.0,2840.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.0,1.0,0.0,1.0,0.0,117.0,0.0,0.0,0.0,-2.5,84866.0,1.5,0.0,0.5,-2.5,1.5,0.0,100.0,0.0,0.0,68.0,5.0
2987001,86401.0,29.0,27.5,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,59.0,12.0,0.0,10.5,0.0,23.5,0.0,0.0,6.5,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5155.0,2840.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-2.5,84866.0,1.5,0.0,0.5,-2.5,1.5,0.0,100.0,0.0,0.0,29.0,0.0
2987002,86469.0,59.0,287.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,59.0,10.5,0.0,10.5,0.0,23.5,0.0,0.0,315.0,0.0,0.0,0.0,315.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5155.0,2840.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-2.5,84866.0,1.5,0.0,0.5,-2.5,1.5,0.0,100.0,0.0,1.0,59.0,0.0
2987003,86499.0,50.0,68.5,2.0,5.0,0.0,0.0,4.0,0.0,1.0,1.0,0.0,25.0,1.0,112.0,112.0,0.0,94.0,0.0,0.0,83.0,0.0,84.0,19.5,0.0,0.0,0.0,111.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,10.0,1758.0,354.0,135.0,0.0,0.0,0.0,2577.5,1370.0,1.5,2.5,0.5,1.0,1.0,2.5,0.5,0.5,0.5,1.0,1.0,83.107697,45.163952,45.163952,45.163952,45.163952,37.943748,37.943748,1.5,2.5,1.0,2.5,0.5,0.5,0.5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,83.107697,83.107697,83.107697,45.163952,45.163952,0.0,37.943748,0.0,0.0,0.0,1.0,0.0,1758.0,354.0,135.0,0.0,-5.0,206731.5,0.0,0.0,0.0,-3.0,0.0,0.0,100.0,0.0,1.0,50.0,0.0
2987004,86506.0,50.0,161.5,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,66.5,20.5,0.0,20.5,17.5,110.0,0.0,0.0,157.5,17.5,159.5,0.0,157.5,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5155.0,2840.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,70787.0,0.0,0.0,0.5,-0.5,0.5,0.0,100.0,0.0,1.0,50.0,0.0


In [42]:
X_train[numf] = knn2.transform(X_train[numf])
X_train.to_pickle('train_clean_knn2.pkl')

In [43]:
X_train.head()

,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,P_emaildomain,R_emaildomain,C1,C2,C4,C5,C6,C7,C9,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V3,V4,V6,V8,V11,V13,V18,V20,V30,V36,V37,V40,V44,V47,V48,V52,V54,V60,V70,V76,V78,V80,V81,V82,V87,V91,V99,V127,V130,V131,V139,V147,V156,V165,V166,V169,V171,V173,V174,V175,V176,V180,V183,V185,V195,V198,V203,V205,V207,V209,V210,V215,V216,V220,V221,V223,V230,V234,V235,V238,V240,V241,V250,V252,V257,V258,V260,V262,V263,V264,V265,V268,V271,V274,V277,V282,V283,V289,V291,V303,V307,V310,V312,V315,id_01,id_02,id_03,id_04,id_05,id_06,id_09,id_10,id_11,id_12,id_13,id_15,id_16,id_17,id_19,id_20,id_28,id_29,id_30,id_33,id_35,id_36,id_37,DeviceType,DeviceInfo,TransactionHR,TransactionMT,TransactionDW,dollars,cents,P_parent_domain,P_domain_name,P_top_level_domain,card1_div_1000,card2_div_10,R_parent_domain,R_domain_name,R_top_level_domain,device_name,device_version,os_name,os_version
TransactionID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2987000,86400.0,68.5,4,3417,500,42,1,38,1,166,65,19.0,31,31,1.0,1.0,0.0,0.0,1.0,0.0,1.0,2.0,0.0,1.0,1.0,14.0,59.0,13.0,0.0,10.5,0.0,23.5,0.0,13.0,13.0,0.0,0.0,0.0,0.0,1,1,1,2,0,1,2,2,2,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.5,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,5155.0,2840.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.0,1.0,0.0,1.0,0.0,117.0,0.0,0.0,0.0,-2.5,84866.0,1.5,0.0,0.5,-2.5,1.5,0.0,100.0,2,54,3,2,104,522,394,2,2,74,260,2,2,2,1,1735,0.0,0.0,5,68.0,5.0,16,24,7,4,51,16,24,8,13,293,7,68
2987001,86401.0,29.0,4,7922,303,42,2,2,1,173,65,27.5,16,31,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,59.0,12.0,0.0,10.5,0.0,23.5,0.0,0.0,6.5,0.0,0.0,0.0,0.0,2,2,2,0,1,1,2,2,2,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5155.0,2840.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-2.5,84866.0,1.5,0.0,0.5,-2.5,1.5,0.0,100.0,2,54,3,2,104,522,394,2,2,74,260,2,2,2,1,1735,0.0,0.0,5,29.0,0.0,12,15,2,10,30,16,24,8,13,293,7,68
2987002,86469.0,59.0,4,9383,389,42,4,58,2,178,65,287.0,36,31,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,59.0,10.5,0.0,10.5,0.0,23.5,0.0,0.0,315.0,0.0,0.0,0.0,315.0,1,1,1,0,0,0,0,0,0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5155.0,2840.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-2.5,84866.0,1.5,0.0,0.5,-2.5,1.5,0.0,100.0,2,54,3,2,104,522,394,2,2,74,260,2,2,2,1,1735,0.0,1.0,5,59.0,0.0,15,28,2,12,39,16,24,8,13,293,7,68
2987003,86499.0,50.0,4,6991,466,42,2,14,2,282,65,68.5,54,31,2.0,5.0,0.0,0.0,4.0,0.0,1.0,1.0,0.0,25.0,1.0,112.0,112.0,0.0,94.0,0.0,0.0,83.0,0.0,84.0,19.5,0.0,0.0,0.0,111.0,2,2,2,0,1,0,2,2,2,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,10.0,1758.0,354.0,135.0,0.0,0.0,0.0,2577.5,1370.0,1.5,2.5,0.5,1.0,1.0,2.5,0.5,0.5,0.5,1.0,1.0,83.107697,45.163952,45.163952,45.163952,45.163952,37.943748,37.943748,1.5,2.5,1.0,2.5,0.5,0.5,0.5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,83.107697,83.107697,83.107697,45.163952,45.163952,0.0,37.9437

In [45]:
train_data = pd.concat([X_train, y_train], axis=1)

In [46]:
train_data.shape

(590540, 176)

In [49]:
train_data.to_pickle('train_clean_knn2.pkl')

In [81]:
X_test.shape

(506691, 175)

In [84]:
X_test[numf] = knn2_test.transform(X_test[numf])
X_test.to_pickle('test_clean_knn2.pkl')

In [85]:
X_test.head()

,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,P_emaildomain,R_emaildomain,C1,C2,C4,C5,C6,C7,C9,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V3,V4,V6,V8,V11,V13,V18,V20,V30,V36,V37,V40,V44,V47,V48,V52,V54,V60,V70,V76,V78,V80,V81,V82,V87,V91,V99,V127,V130,V131,V139,V147,V156,V165,V166,V169,V171,V173,V174,V175,V176,V180,V183,V185,V195,V198,V203,V205,V207,V209,V210,V215,V216,V220,V221,V223,V230,V234,V235,V238,V240,V241,V250,V252,V257,V258,V260,V262,V263,V264,V265,V268,V271,V274,V277,V282,V283,V289,V291,V303,V307,V310,V312,V315,id_01,id_02,id_03,id_04,id_05,id_06,id_09,id_10,id_11,id_12,id_13,id_15,id_16,id_17,id_19,id_20,id_28,id_29,id_30,id_33,id_35,id_36,id_37,DeviceType,DeviceInfo,TransactionHR,TransactionMT,TransactionDW,dollars,cents,card1_div_1000,card2_div_10,P_parent_domain,P_domain_name,P_top_level_domain,R_parent_domain,R_domain_name,R_top_level_domain,device_name,device_version,os_name,os_version
TransactionID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3663549,18403224.0,31.95,4,365,10,42,4,108,2,52,65,1.0,16,31,6.0,6.0,0.0,3.0,4.0,0.0,6.0,5.0,1.0,115.0,6.0,419.0,419.0,27.0,398.0,27.0,317.0,11.0,0.0,418.0,203.0,0.0,0.0,0.0,409.0,1,1,0,3,2,0,1,1,1,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,47.950001,47.950001,0.000000,0.5,0.0,0.5,0.0,0.0,0.0,1.0,0.0,0.5,0.5,1.0,0.0,0.0,0.0,1.0,1.0,20.0,0.0,0.0,20.0,20.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,47.950001,47.950001,0.000000,0.000000,-22.5,141934.5,0.0,0.0,0.0,0.0,0.0,0.0,100.0,2,54,3,2,104,522,394,2,2,74,260,2,2,2,1,1735,0.0,0.0,0,31.0,95.0,1,1,12,15,2,16,24,8,13,293,7,68
3663550,18403263.0,49.00,4,9099,10,42,4,108,2,151,65,4.0,2,31,3.0,2.0,0.0,0.0,1.0,0.0,2.0,1.0,1.0,12.0,2.0,149.0,149.0,7.0,634.0,7.0,317.0,11.0,0.0,231.0,634.0,283.5,0.0,0.0,634.0,1,0,0,0,2,0,2,2,2,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,4.0,280.000000,280.000000,77.000000,0.5,0.0,0.5,0.0,0.0,0.0,1.0,0.0,0.5,0.5,1.0,0.0,0.0,0.0,1.0,1.0,20.0,0.0,0.0,20.0,20.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,280.000000,280.000000,77.000000,0.000000,-22.5,141934.5,0.0,0.0,0.0,0.0,0.0,0.0,100.0,2,54,3,2,104,522,394,2,2,74,260,2,2,2,1,1735,0.0,1.0,0,49.0,0.0,12,1,2,2,2,16,24,8,13,293,7,68
3663551,18403310.0,171.00,4,9245,473,42,4,108,2,280,65,2635.0,19,31,2.0,2.0,0.0,0.0,5.0,0.0,4.0,2.0,0.0,22.0,2.0,137.0,137.0,10.0,97.0,10.0,317.0,11.0,0.0,136.0,136.0,0.0,0.0,0.0,97.0,1,1,0,0,0,0,0,0,0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,1.0,1.0,0.0,3.0,968.000000,705.000000,0.000000,1.5,0.0,0.5,50.0,0.0,0.0,1.0,0.0,0.5,0.5,1.0,0.0,0.0,0.0,1.0,1.0,20.0,0.0,0.0,20.0,20.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1321.000000,1058.000000,0.000000,0.000000,-22.5,141934.5,0.0,0.0,0.0,0.0,0.0,0.0,100.0,2,54,3,2,104,522,394,2,2,74,260,2,2,2,1,1735,0.0,1.0,0,171.0,0.0,12,47,15,17,2,16,24,8,13,293,7,68
3663552,18403310.0,284.95,4,870,259,42,4,58,2,79,65,17.0,16,31,5.0,2.0,0.0,1.0,1.0,0.0,2.0,2.0,0.0,7.0,4.0,42.0,42.0,41.0,242.0,41.0,0.0,11.0,0.0,242.0,242.0,0.0,0.0,0.0,242.0,1,1,1,3,2,1,2,2,2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,2.0,2.0,0.0,2.0,2.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.000000,0.000000,0.000000,0.5,0.0,0.5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.000000,0.000000,0.000000,282.540009,-25.0,232750.0,0.0,0.0,0.5,0.0,0.0,0.0,100.

In [86]:
X_test.isna().sum()

TransactionDT         0
TransactionAmt        0
ProductCD             0
card1                 0
card2                 0
card3                 0
card4                 0
card5                 0
card6                 0
addr1                 0
addr2                 0
dist1                 0
P_emaildomain         0
R_emaildomain         0
C1                    0
C2                    0
C4                    0
C5                    0
C6                    0
C7                    0
C9                    0
C11                   0
C12                   0
C13                   0
C14                   0
D1                    0
D2                    0
D3                    0
D4                    0
D5                    0
D6                    0
D8                    0
D9                    0
D10                   0
D11                   0
D12                   0
D13                   0
D14                   0
D15                   0
M1                    0
M2                    0
M3              

In [87]:
X_test.to_pickle('test_clean_knn2.pkl')